# imports

In [1]:
import os, json
import openai
from dotenv import dotenv_values, load_dotenv
from openai.types.chat.chat_completion import ChatCompletion
from IPython.display import display, Markdown


# load api key

In [2]:
load_dotenv()
client:openai.OpenAI = openai.OpenAI()

# functions
## openai helper function

In [3]:
def get_llm_response(client:openai.OpenAI, prompt:str, model:str='gpt-4o-mini', return_dict_body:bool=False, return_response_body:bool=False) -> str | dict | ChatCompletion:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    
    # print(type(response))
    
    if return_response_body:
        return response
    if return_dict_body:
        return response.model_dump()
    return response.choices[0].message.content


## prompt template interpolate function

In [4]:
def get_colors_prompt(prompt:str) -> str:
    return f'''
    You are a color palette generating assistant that responds to text prompts for color palettes
    Your should generate color palettes that fit the theme, mood, or instructions in the prompt.
    The palettes should be between 2 and 8 colors, the more the better.

    Q: Convert the following verbal description of a color palette into a list of colors: The Mediterranean Sea
    A: ["#006699", "#66CCCC", "#F0E68C", "#008000", "#F08080"]

    Q: Convert the following verbal description of a color palette into a list of colors: sage, nature, earth
    A: ["#EDF1D6", "#9DC08B", "#609966", "#40513B"]


    Desired Format: just a JSON array of hexadecimal color codes, nothing else before or after

    Q: Convert the following verbal description of a color palette into a list of colors: {prompt}
    A:    
    '''


## notebook markdown display helper

In [5]:
def display_colors(colors, width:int=4):
    display(
        Markdown(
            ' '.join(
                f'<span style="color: {color}">{chr(9608) * width}</span>'
                for color in colors
            )
        )
    )

# main
## get json color array

In [6]:
response = get_llm_response(
    client, 
    get_colors_prompt('blue ocean')
)
response

'["#003F5C","#6B8EAD","#A8DADC","#F1FAEE","#457B9D"]'

## convert json to python array

In [7]:
colors = json.loads(response)
colors

['#003F5C', '#6B8EAD', '#A8DADC', '#F1FAEE', '#457B9D']

## display result

In [8]:
display_colors(colors)

<span style="color: #003F5C">████</span> <span style="color: #6B8EAD">████</span> <span style="color: #A8DADC">████</span> <span style="color: #F1FAEE">████</span> <span style="color: #457B9D">████</span>

# macro helper functions

In [9]:
def get_color_palette(client:openai.OpenAI, prompt:str) -> list:
    userprompt = get_colors_prompt(prompt)
    response = get_llm_response(client, userprompt)
    colors = json.loads(response)
    return colors


In [10]:
from json import JSONDecodeError


def show_palette(client:openai.OpenAI, prompt:str):
    try:
        colors = get_color_palette(client, prompt)
        display_colors(colors)
        print(colors)
    except JSONDecodeError as jde:
        print(f'something wrong with llm response format: {response!r}')

In [11]:
show_palette(client, 'ocean abyss')

<span style="color: #001F3F">████</span> <span style="color: #003D5B">████</span> <span style="color: #5B8FF3">████</span> <span style="color: #2E8B57">████</span> <span style="color: #A0DAB9">████</span> <span style="color: #B8E3D2">████</span>

['#001F3F', '#003D5B', '#5B8FF3', '#2E8B57', '#A0DAB9', '#B8E3D2']


In [12]:
show_palette(client, 'cyber terminal with green chars')

something wrong with llm response format: '["#003F5C","#6B8EAD","#A8DADC","#F1FAEE","#457B9D"]'


In [13]:
show_palette(client, 'spongebob')

<span style="color: #F9E21B">████</span> <span style="color: #F4B400">████</span> <span style="color: #D20A0B">████</span> <span style="color: #A200DD">████</span> <span style="color: #0077C8">████</span>

['#F9E21B', '#F4B400', '#D20A0B', '#A200DD', '#0077C8']


In [14]:
show_palette(client, 'superman')

<span style="color: #C8102E">████</span> <span style="color: #003DA5">████</span> <span style="color: #FEDD00">████</span> <span style="color: #FFFFFF">████</span> <span style="color: #000000">████</span>

['#C8102E', '#003DA5', '#FEDD00', '#FFFFFF', '#000000']


In [15]:
show_palette(client, 'european tabby cat white gray and black')

<span style="color: #FFFFFF">████</span> <span style="color: #B0B3B7">████</span> <span style="color: #7D7D7D">████</span> <span style="color: #3B3B3B">████</span>

['#FFFFFF', '#B0B3B7', '#7D7D7D', '#3B3B3B']
